In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dataset-gob-tarea/README.md
/kaggle/input/dataset-gob-tarea/.DS_Store
/kaggle/input/dataset-gob-tarea/input/atus_anual_2021.csv
/kaggle/input/dataset-gob-tarea/input/atus_anual_2022.csv
/kaggle/input/dataset-gob-tarea/input/atus_anual_2020.csv
/kaggle/input/dataset-gob-tarea/input/.DS_Store


## Cargue en un Dataframe los datos de los accidentes de los últimos 3 años

In [2]:
# Leer los archivos para luego unirlos, los primeros 2 tenian problemas con el indice
df20 = pd.read_csv('/kaggle/input/dataset-gob-tarea/input/atus_anual_2020.csv', index_col=False)
df21 = pd.read_csv('/kaggle/input/dataset-gob-tarea/input/atus_anual_2021.csv', index_col=False)
df22 = pd.read_csv('/kaggle/input/dataset-gob-tarea/input/atus_anual_2022.csv')

df = pd.concat([df20, df21, df22])

# Verificar el resultado
print(df.head(2))

   COBERTURA  ID_ENTIDAD  ID_MUNICIPIO  ANIO  MES  ID_HORA  ID_MINUTO  ID_DIA  \
0  Municipal           1             1  2020    1        0          0       1   
1  Municipal           1             1  2020    1        0          0       1   

   DIASEMANA                     URBANA  ... PEATMUERTO PEATHERIDO  \
0  Miercoles  Accidente en intersección  ...          0          0   
1  Miercoles  Accidente en intersección  ...          0          0   

   CICLMUERTO  CICLHERIDO  OTROMUERTO  OTROHERIDO  NEMUERTO  NEHERIDO  \
0           0           0           0           0         0         0   
1           0           0           0           0         0         0   

      CLASACC             ESTATUS  
0  Sólo daños  Cifras Definitivas  
1  Sólo daños  Cifras Definitivas  

[2 rows x 45 columns]


## Agregue una columna tipo date que englobe las columnas correspondientes a Año, Mes, Dia, Hora y Minuto.

In [3]:
# Especificar qué columnas representan cada parte de la fecha y la hora
#df['Fecha'] = pd.to_datetime(df[['ANIO', 'MES', 'ID_DIA','ID_HORA','ID_MINUTO']])

# Seleccionar solo la columna 'Fecha'
#sdf = df[['Fecha']]

# Verificar el resultado
#print(sdf.head())

## Agregue una columna para Área cuyos valores serán Urbana o Suburbana dependiendo del área donde ocurrió el accidente.

In [4]:
#Crear la columna "Área" basada en las columnas "URBANA" y "SUBURBANA"
#Establecer inicialmente todos los valores como "Urbana"
df['Area'] = 'Urbana'  

#si urbana tiene ese mensaja entonces el area es en suburbana
df.loc[df['URBANA'] == 'Sin accidente en esta zona', 'Area'] = 'Suburbana' 

# Verificar el resultado
print(df[['Area']].head(121))

          Area
0       Urbana
1       Urbana
2       Urbana
3       Urbana
4       Urbana
..         ...
116     Urbana
117     Urbana
118     Urbana
119  Suburbana
120     Urbana

[121 rows x 1 columns]


## Elimine las columnas URBANA y SUBURBANA

In [5]:
df = df.drop(columns=['URBANA', 'SUBURBANA'])

# Verificar el resultado
print(df.head())

   COBERTURA  ID_ENTIDAD  ID_MUNICIPIO  ANIO  MES  ID_HORA  ID_MINUTO  ID_DIA  \
0  Municipal           1             1  2020    1        0          0       1   
1  Municipal           1             1  2020    1        0          0       1   
2  Municipal           1             1  2020    1        0         10       1   
3  Municipal           1             1  2020    1        2          0       1   
4  Municipal           1             1  2020    1        2         10       1   

   DIASEMANA                         TIPACCID  ...  PEATHERIDO  CICLMUERTO  \
0  Miercoles  Colisión con vehículo automotor  ...           0           0   
1  Miercoles         Colisión con objeto fijo  ...           0           0   
2  Miercoles         Colisión con motocicleta  ...           0           0   
3  Miercoles  Colisión con vehículo automotor  ...           0           0   
4  Miercoles  Colisión con vehículo automotor  ...           0           0   

   CICLHERIDO  OTROMUERTO  OTROHERIDO  NEMUE

## Determine cual municipio de cada estado tuvo más meses sin accidentes

In [6]:
# Agrupar por entidad, municipio y mes, y contar el número de registros para cada grupo
agrupado = df.groupby(['ID_ENTIDAD', 'ID_MUNICIPIO', 'MES']).size().reset_index(name='count')

# Encontrar el municipio de cada estado que tuvo la menor cantidad de registros 
municipio_con_menos_meses_por_estado = agrupado.loc[agrupado.groupby('ID_ENTIDAD')['count'].idxmin()]

# Verificar el resultado
print(municipio_con_menos_meses_por_estado)

       ID_ENTIDAD  ID_MUNICIPIO  MES  count
36              1             4    1      3
157             2             3    2     29
241             3             9    2     15
384             4            12    1      1
408             5             1    1      3
928             6             6    5      3
2460            7           125    1      1
2496            8             3    1      3
3099            9             8    4     17
3228           10             3    6      1
3589           11             6    1      3
4429           12            31    1      1
5017           13             1    1      3
6040           14             2    4      3
7526           15             1    2      1
8830           16            29    1      2
10270          17            36    1      2
10378          18             9    1      1
10522          19             1    1      3
11134          20             1    1      3
17974          21             1    1      3
20722          22            13 

## Resuma los accidentes del último año por tipo y que porcentaje representan

In [7]:
#usar solo el archivo del ultimo año
df1 = pd.read_csv('/kaggle/input/dataset-gob-tarea/input/atus_anual_2022.csv')

# Contar el número de accidentes por tipo
resumen_accidentes = df1['TIPACCID'].value_counts().reset_index()
resumen_accidentes.columns = ['Tipo de accidente', 'Cantidad']

# Calcular el porcentaje que representa cada tipo de accidente en el total de accidentes
total_accidentes = resumen_accidentes['Cantidad'].sum()
resumen_accidentes['Porcentaje'] = (resumen_accidentes['Cantidad'] / total_accidentes) * 100

# Verificar el resultado
print(resumen_accidentes)

                        Tipo de accidente  Cantidad  Porcentaje
0         Colisión con vehículo automotor    229940   58.552112
1                Colisión con motocicleta     53629   13.656133
2                Colisión con objeto fijo     47033   11.976522
3                        Certificado cero     15479    3.941585
4   Colisión con peatón (atropellamiento)     11066    2.817855
5                               Volcadura     10911    2.778386
6                       Salida del camino     10857    2.764635
7                                    Otro      6325    1.610603
8                   Colisión con ciclista      4048    1.030786
9                       Caída de pasajero      1784    0.454279
10                    Colisión con animal      1022    0.260243
11                               Incendio       348    0.088615
12               Colisión con ferrocarril       268    0.068244


## Obtenga la cantidad de muertos y lesionados de los últimos 3 años

In [8]:
# Sumar las columnas que indican el número de muertos
muertos = df[['CONDMUERTO', 'PASAMUERTO', 'PEATMUERTO', 'CICLMUERTO', 'OTROMUERTO', 'NEMUERTO']].sum().sum()

# Sumar las columnas que indican el número de lesionados
lesionados = df[['CONDHERIDO', 'PASAHERIDO', 'PEATHERIDO', 'CICLHERIDO', 'OTROHERIDO', 'NEHERIDO']].sum().sum()

# Verificar el resultado
print("Cantidad de muertos en los últimos 3 años:", muertos)
print("Cantidad de lesionados en los últimos 3 años:", lesionados)

Cantidad de muertos en los últimos 3 años: 13408
Cantidad de lesionados en los últimos 3 años: 245902


## Obtenga la cantidad de muertos y lesionados de los últimos 3 años

In [9]:
# Contar el número de accidentes por año
accidentes_por_anio = df['ANIO'].value_counts().sort_index()

# Calcular la variación porcentual entre cada par de años consecutivos
variacion_porcentual = (accidentes_por_anio.pct_change() * 100).fillna(0)

# Verificar el resultado
print("Número de accidentes por año:")
print(accidentes_por_anio)
print("\nVariación porcentual respecto al año anterior:")
print(variacion_porcentual)

Número de accidentes por año:
ANIO
2020    318046
2021    356315
2022    392710
Name: count, dtype: int64

Variación porcentual respecto al año anterior:
ANIO
2020     0.000000
2021    12.032536
2022    10.214277
Name: count, dtype: float64


## cada año los accidentes aumentan alrededor de un 10%.